Projekt - DEA - Graniczna Analiza Danych

In [1]:
# importy i input
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pulp import *
from tabulate import tabulate

inputs = pd.read_csv('inputs.csv', index_col=0, delimiter=';')
outputs = pd.read_csv('outputs.csv', index_col=0, delimiter=';')
samples = pd.read_csv('samples_homework.csv', delimiter=';')

In [2]:
efficiency_scores = {}
reference_units = {}
adjustments = {}
super_efficiency_scores = {}
cross_efficiency_scores = {DMU: [] for DMU in inputs.index}
expected_efficiency = {}

In [3]:
# Obliczanie efektywności i superefektywności oraz HCU
for DMUo in inputs.index:
    # Model klasyczny (efektywność)
    problem = LpProblem(f"Efficiency_{DMUo}", LpMinimize)
    lambdas = LpVariable.dicts("Lambda", (i for i in inputs.index), lowBound=0)
    theta = LpVariable("theta", lowBound=0, upBound=1)

    problem += theta

    for input_metric in inputs.columns:
        problem += lpSum([lambdas[i] * inputs.loc[i, input_metric] for i in inputs.index]) <= theta * inputs.loc[DMUo, input_metric]

    for output_metric in outputs.columns:
        problem += lpSum([lambdas[i] * outputs.loc[i, output_metric] for i in inputs.index]) >= outputs.loc[DMUo, output_metric]

    problem.solve(PULP_CBC_CMD(msg=False))

    if LpStatus[problem.status] == 'Optimal':
        efficiency_score = value(theta)
        efficiency_scores[DMUo] = efficiency_score

        if efficiency_score < 1:
            # Zapisz jednostki odniesienia (HCU)
            reference_units[DMUo] = {}
            adjustments[DMUo] = {}
            for i in inputs.index:
                if lambdas[i].varValue > 0:
                    reference_units[DMUo][i] = lambdas[i].varValue

            # Oblicz poprawki
            for input_metric in inputs.columns:
                optimal_input = sum(lambdas[i].varValue * inputs.loc[i, input_metric] for i in inputs.index)
                adjustments[DMUo][input_metric] = inputs.loc[DMUo, input_metric] - optimal_input

            # Superefektywność = efektywność (dla nieefektywnych jednostek)
            super_efficiency_scores[DMUo] = efficiency_score

        else:
            # Model superefektywności (tylko dla efektywnych jednostek)
            super_problem = LpProblem(f"SuperEfficiency_{DMUo}", LpMinimize)
            # tylko inne jednostki
            peer_dmus = [i for i in inputs.index if i != DMUo]
            super_lambdas = LpVariable.dicts("SuperLambda", peer_dmus, lowBound=0)
            super_theta = LpVariable("super_theta", lowBound=0)

            super_problem += super_theta

            # Ograniczenia: wejścia — pomnóż wejścia peerów przez lambdy, ale pomiń DMUo!
            for input_metric in inputs.columns:
                super_problem += lpSum([
                    super_lambdas[i] * inputs.loc[i, input_metric] for i in peer_dmus
                ]) <= super_theta * inputs.loc[DMUo, input_metric]

            # Ograniczenia: wyjścia — peerzy muszą osiągać co najmniej to co DMUo
            for output_metric in outputs.columns:
                super_problem += lpSum([
                    super_lambdas[i] * outputs.loc[i, output_metric] for i in peer_dmus
                ]) >= outputs.loc[DMUo, output_metric]

            super_problem.solve(PULP_CBC_CMD(msg=False))

            if LpStatus[super_problem.status] == 'Optimal':
                theta_val = value(super_theta)
                if theta_val and theta_val > 1e-6:
                    super_eff = 1 / theta_val
                    if super_eff >= 1:
                        super_efficiency_scores[DMUo] = super_eff
                    else:
                        print(f"Warning: Superefektywność dla {DMUo} < 1 ({super_eff:.3f}) — to nie powinno się zdarzyć! - fix w następnej komórce")
                        super_efficiency_scores[DMUo] = np.nan
                else:
                    super_efficiency_scores[DMUo] = np.nan
            else:
                print(f"Superefektywność: problem dla {DMUo} jest nierozwiązywalny")
    else:
        print(f"Efektywność: problem dla {DMUo} jest nierozwiązywalny")

In [4]:
# Obliczanie superefektywności - poprawione?
for DMUo in inputs.index:
    super_problem = LpProblem(f"SEfficiency_{DMUo}", LpMinimize)

    # Usuwamy DMUo z danych referencyjnych
    super_inputs = inputs.drop(index=DMUo)
    super_outputs = outputs.drop(index=DMUo)

    # Zmienna decyzyjna: lambda tylko dla pozostałych
    s_lambdas = LpVariable.dicts("Lambda", super_inputs.index, lowBound=0)
    s_theta = LpVariable("theta", lowBound=0)

    super_problem += s_theta  # minimalizacja theta

    # Ograniczenia wejściowe
    for input_metric in super_inputs.columns:
        super_problem += lpSum([s_lambdas[i] * super_inputs.loc[i, input_metric] for i in super_inputs.index]) <= s_theta * inputs.loc[DMUo, input_metric]

    # Ograniczenia wyjściowe
    for output_metric in super_outputs.columns:
        super_problem += lpSum([s_lambdas[i] * super_outputs.loc[i, output_metric] for i in super_outputs.index]) >= outputs.loc[DMUo, output_metric]

    # Rozwiązanie
    super_problem.solve(PULP_CBC_CMD(msg=False))

    if LpStatus[super_problem.status] == 'Optimal':
        super_efficiency_score = value(s_theta)
        super_efficiency_scores[DMUo] = super_efficiency_score
    else:
        print(f"Superefektywność: problem dla {DMUo} jest nierozwiązywalny")


In [5]:
# Tutaj robię tabelki z tego wyżej i wyświetlam tak, żeby to wygodnie importować do sprawka.
# Tabela efektywności
df_efficiency = pd.DataFrame.from_dict(efficiency_scores, orient='index', columns=['Efektywność'])
df_efficiency.index.name = 'Lotnisko'

# Tabela superefektywności
df_super = pd.DataFrame.from_dict(super_efficiency_scores, orient='index', columns=['Superefektywność'])
df_super.index.name = 'Lotnisko'

# Tabela poprawek (adjustments)
df_adjustments = pd.DataFrame.from_dict(adjustments, orient='index')
df_adjustments.index.name = 'Lotnisko'

# Oblicz rzeczywiste wartości wejściowe dla HCU (hipotetycznej jednostki odniesienia)
hcu_inputs = {}

for dmu, lambda_dict in reference_units.items():
    hcu_inputs[dmu] = {}
    for input_metric in inputs.columns:
        hcu_inputs[dmu][input_metric] = sum(
            lambda_dict.get(k, 0) * inputs.loc[k, input_metric] for k in inputs.index
        )

# Przekształć do DataFrame
df_hcu_inputs = pd.DataFrame.from_dict(hcu_inputs, orient='index')
df_hcu_inputs.index.name = "Lotnisko"
df_hcu_inputs.columns = [f"{col}_HCU" for col in df_hcu_inputs.columns]
df_adjustments.columns = [f"{col}_Δ" for col in df_adjustments.columns]

# Połącz dane
df_hcu_full = df_hcu_inputs.join(df_adjustments, how="left")


# Złączenie efektywności i superefektywności
df_zad2 = df_hcu_inputs.join(df_adjustments, how='left')


# Wyświetlenie
print("Tabela efektywności:")
print(tabulate(df_efficiency.reset_index(), headers='keys', tablefmt='github'))

print("\nTabela superefektywności:")
print(tabulate(df_super.reset_index(), headers='keys', tablefmt='github'))

print("\nTabela jednostek odniesienia (HCU):")
print(tabulate(df_hcu_inputs.reset_index(), headers='keys', tablefmt='github'))

print("\nTabela poprawek:")
print(tabulate(df_adjustments.reset_index(), headers='keys', tablefmt='github'))

print("\nTabela HCU i poprawki:")
print(tabulate(df_zad2.reset_index(), headers='keys', tablefmt='github'))

Tabela efektywności:
|    | Lotnisko   |   Efektywność |
|----|------------|---------------|
|  0 | WAW        |      1        |
|  1 | KRK        |      1        |
|  2 | KAT        |      0.591209 |
|  3 | WRO        |      1        |
|  4 | POZ        |      0.799801 |
|  5 | LCJ        |      0.300036 |
|  6 | GDN        |      1        |
|  7 | SZZ        |      0.270787 |
|  8 | BZG        |      1        |
|  9 | RZE        |      0.409183 |
| 10 | IEG        |      0.258475 |

Tabela superefektywności:
|    | Lotnisko   |   Superefektywność |
|----|------------|--------------------|
|  0 | WAW        |           2.27795  |
|  1 | KRK        |           1.12378  |
|  2 | KAT        |           0.591209 |
|  3 | WRO        |           1.03995  |
|  4 | POZ        |           0.799801 |
|  5 | LCJ        |           0.300036 |
|  6 | GDN        |           2        |
|  7 | SZZ        |           0.270787 |
|  8 | BZG        |           1.74593  |
|  9 | RZE        |           0.4

In [6]:
# importy do LaTeXa do sprawka ;)
print(df_efficiency.to_latex(
    header=["Efektywność"],
    index_names=True,
    column_format="c|c",
    float_format="%.3f"
))

\begin{tabular}{c|c}
\toprule
 & Efektywność \\
Lotnisko &  \\
\midrule
WAW & 1.000 \\
KRK & 1.000 \\
KAT & 0.591 \\
WRO & 1.000 \\
POZ & 0.800 \\
LCJ & 0.300 \\
GDN & 1.000 \\
SZZ & 0.271 \\
BZG & 1.000 \\
RZE & 0.409 \\
IEG & 0.258 \\
\bottomrule
\end{tabular}



In [7]:
print(df_zad2.to_latex(
    index_names=True,
    column_format="c|" + "c" * len(df_zad2.columns),
    float_format="%.2f"
).replace("\\toprule", "\\hline")
 .replace("\\midrule", "\\hline")
 .replace("\\bottomrule", "\\hline"))


\begin{tabular}{c|cccccccc}
\hline
 & i1_HCU & i2_HCU & i3_HCU & i4_HCU & i1_Δ & i2_Δ & i3_Δ & i4_Δ \\
Lotnisko &  &  &  &  &  &  &  &  \\
\hline
KAT & 2.13 & 18.92 & 33.94 & 4.40 & 1.47 & 13.08 & 23.46 & 6.10 \\
POZ & 1.20 & 8.00 & 19.20 & 1.93 & 0.30 & 2.00 & 4.80 & 2.07 \\
LCJ & 0.18 & 2.78 & 7.20 & 0.47 & 0.42 & 9.22 & 16.80 & 3.43 \\
SZZ & 0.19 & 2.71 & 6.96 & 0.47 & 0.51 & 7.29 & 18.74 & 1.43 \\
RZE & 0.25 & 2.46 & 4.62 & 0.54 & 0.35 & 3.54 & 6.68 & 2.16 \\
IEG & 0.03 & 0.39 & 1.11 & 0.06 & 0.07 & 9.61 & 62.29 & 2.94 \\
\hline
\end{tabular}



In [8]:
print(df_super.to_latex(
    header=["Superefektywność"],
    index_names=True,
    column_format="c|c",
    float_format="%.3f"
))


\begin{tabular}{c|c}
\toprule
 & Superefektywność \\
Lotnisko &  \\
\midrule
WAW & 2.278 \\
KRK & 1.124 \\
KAT & 0.591 \\
WRO & 1.040 \\
POZ & 0.800 \\
LCJ & 0.300 \\
GDN & 2.000 \\
SZZ & 0.271 \\
BZG & 1.746 \\
RZE & 0.409 \\
IEG & 0.258 \\
\bottomrule
\end{tabular}



In [9]:
# Assuming the necessary data frames 'inputs' and 'outputs' are already defined
cross_eff_matrix = pd.DataFrame(index=inputs.index, columns=inputs.index, dtype=float)

# Oblicz efektywność krzyżową
for evaluator in inputs.index:  # DMU optymalizujący swoje wagi (dostarczający je innym)
    # Zmienna decyzyjna: wagi dla wyjść i wejść
    u = LpVariable.dicts("u", outputs.columns, lowBound=0)
    v = LpVariable.dicts("v", inputs.columns, lowBound=0)

    # Problem maksymalizacji (dualny model CCR - input-oriented)
    prob = LpProblem(f"Efficiency_{evaluator}", LpMaximize)

    # Funkcja celu: suma ważonych wyjść DMU oceniającego
    prob += lpSum([u[r] * outputs.loc[evaluator, r] for r in outputs.columns])

    # Ograniczenie: suma ważonych wejść = 1
    prob += lpSum([v[i] * inputs.loc[evaluator, i] for i in inputs.columns]) == 1

    # Ograniczenia dla pozostałych DMU – efektywność nie może przekraczać 1 przy tych wagach
    for dmu in inputs.index:
        prob += lpSum([u[r] * outputs.loc[dmu, r] for r in outputs.columns]) <= lpSum([v[i] * inputs.loc[dmu, i] for i in inputs.columns])

    # Rozwiązanie
    prob.solve(PULP_CBC_CMD(msg=False))

    if LpStatus[prob.status] == 'Optimal':
        for evaluated in inputs.index:  # DMU oceniane przy wagach z evaluator
            numerator = sum(value(u[r]) * outputs.loc[evaluated, r] for r in outputs.columns)
            denominator = sum(value(v[i]) * inputs.loc[evaluated, i] for i in inputs.columns)
            cross_eff_matrix.loc[evaluated, evaluator] = numerator / denominator
    else:
        print(f"Problem dla {evaluator} nie jest rozwiązywalny.")

# Obliczenie średnich ocen krzyżowych
cross_eff_matrix['CRavg'] = cross_eff_matrix.mean(axis=1)

# Format the table for display
table_format = cross_eff_matrix.reset_index().to_markdown(index=False)
print(table_format)


| index   |        WAW |        KRK |        KAT |        WRO |        POZ |       LCJ |      GDN |       SZZ |       BZG |        RZE |      IEG |    CRavg |
|:--------|-----------:|-----------:|-----------:|-----------:|-----------:|----------:|---------:|----------:|----------:|-----------:|---------:|---------:|
| WAW     | 1          | 1          | 0.912758   | 1          | 1          | 0.595046  | 0.452381 | 1         | 0.595046  | 0.902747   | 0.523406 | 0.816489 |
| KRK     | 0.806201   | 1          | 1          | 1          | 1          | 0.49107   | 0.467742 | 0.755486  | 0.49107   | 0.996316   | 0.427829 | 0.766883 |
| KAT     | 0.468961   | 0.575065   | 0.591209   | 0.562943   | 0.562943   | 0.277842  | 0.333333 | 0.371452  | 0.277842  | 0.591203   | 0.248352 | 0.441922 |
| WRO     | 0.747692   | 0.964851   | 1          | 1          | 1          | 0.604639  | 0.5      | 0.856267  | 0.604639  | 1          | 0.531073 | 0.800833 |
| POZ     | 0.715515   | 0.793061   | 0.77369 

In [10]:
print(cross_eff_matrix.round(3).to_latex(
    header=True,
    index_names=True,
    column_format='c|' + 'c' * (len(cross_eff_matrix.columns)),
    float_format="%.3f"
))


\begin{tabular}{c|cccccccccccc}
\toprule
 & WAW & KRK & KAT & WRO & POZ & LCJ & GDN & SZZ & BZG & RZE & IEG & CRavg \\
\midrule
WAW & 1.000 & 1.000 & 0.913 & 1.000 & 1.000 & 0.595 & 0.452 & 1.000 & 0.595 & 0.903 & 0.523 & 0.816 \\
KRK & 0.806 & 1.000 & 1.000 & 1.000 & 1.000 & 0.491 & 0.468 & 0.755 & 0.491 & 0.996 & 0.428 & 0.767 \\
KAT & 0.469 & 0.575 & 0.591 & 0.563 & 0.563 & 0.278 & 0.333 & 0.371 & 0.278 & 0.591 & 0.248 & 0.442 \\
WRO & 0.748 & 0.965 & 1.000 & 1.000 & 1.000 & 0.605 & 0.500 & 0.856 & 0.605 & 1.000 & 0.531 & 0.801 \\
POZ & 0.716 & 0.793 & 0.774 & 0.800 & 0.800 & 0.512 & 0.433 & 0.737 & 0.512 & 0.770 & 0.458 & 0.664 \\
LCJ & 0.202 & 0.240 & 0.259 & 0.255 & 0.255 & 0.300 & 0.250 & 0.273 & 0.300 & 0.261 & 0.297 & 0.263 \\
GDN & 1.000 & 1.000 & 1.000 & 1.000 & 1.000 & 1.000 & 1.000 & 1.000 & 1.000 & 1.000 & 1.000 & 1.000 \\
SZZ & 0.222 & 0.234 & 0.238 & 0.243 & 0.243 & 0.261 & 0.214 & 0.271 & 0.261 & 0.238 & 0.254 & 0.244 \\
BZG & 0.404 & 0.721 & 0.972 & 0.909 & 0.909 & 1.

In [11]:
bins = [0.0, 0.2, 0.4, 0.6, 0.8, 1.01]  # 1.01 by objąć dokładnie 1.0
labels = ['[0-0.2)', '[0.2-0.4)', '[0.4-0.6)', '[0.6-0.8)', '[0.8-1.0]']

cross_dist = pd.DataFrame(index=cross_eff_matrix.index, columns=labels + ['EE'])

for dmu in cross_eff_matrix.index:
    # Pomijamy self-evaluation
    others = cross_eff_matrix.loc[dmu].drop(index=dmu)

    # Przypisujemy przedziały
    bins_assigned = pd.cut(others, bins=bins, labels=labels, right=False)

    # Obliczamy udziały (czyli % ocen wpadających do danego przedziału)
    distribution = bins_assigned.value_counts(normalize=True, sort=False)

    # Wypełniamy wartości
    for label in labels:
        cross_dist.loc[dmu, label] = round(distribution.get(label, 0), 2)

    # Dodajemy średnią efektywność (EE = CRavg)
    cross_dist.loc[dmu, 'EE'] = round(cross_eff_matrix.loc[dmu].drop(index=dmu).mean(), 3)

# Ustawiamy typy float do prezentacji
cross_dist = cross_dist.astype(float)

print(tabulate(cross_dist.reset_index(), headers='keys', tablefmt='github', floatfmt=".2f"))


|    | index   |   [0-0.2) |   [0.2-0.4) |   [0.4-0.6) |   [0.6-0.8) |   [0.8-1.0] |   EE |
|----|---------|-----------|-------------|-------------|-------------|-------------|------|
|  0 | WAW     |      0.00 |        0.00 |        0.36 |        0.00 |        0.64 | 0.80 |
|  1 | KRK     |      0.00 |        0.00 |        0.36 |        0.18 |        0.45 | 0.75 |
|  2 | KAT     |      0.00 |        0.45 |        0.55 |        0.00 |        0.00 | 0.43 |
|  3 | WRO     |      0.00 |        0.00 |        0.18 |        0.27 |        0.55 | 0.78 |
|  4 | POZ     |      0.00 |        0.00 |        0.36 |        0.64 |        0.00 | 0.65 |
|  5 | LCJ     |      0.00 |        1.00 |        0.00 |        0.00 |        0.00 | 0.26 |
|  6 | GDN     |      0.00 |        0.00 |        0.00 |        0.00 |        1.00 | 1.00 |
|  7 | SZZ     |      0.00 |        1.00 |        0.00 |        0.00 |        0.00 | 0.24 |
|  8 | BZG     |      0.00 |        0.00 |        0.18 |        0.09 |        0.

In [12]:
print(cross_dist.to_latex(index=True, column_format="c|" + "c"*5 + "|c", float_format="%.2f"))

\begin{tabular}{c|ccccc|c}
\toprule
 & [0-0.2) & [0.2-0.4) & [0.4-0.6) & [0.6-0.8) & [0.8-1.0] & EE \\
\midrule
WAW & 0.00 & 0.00 & 0.36 & 0.00 & 0.64 & 0.80 \\
KRK & 0.00 & 0.00 & 0.36 & 0.18 & 0.45 & 0.75 \\
KAT & 0.00 & 0.45 & 0.55 & 0.00 & 0.00 & 0.43 \\
WRO & 0.00 & 0.00 & 0.18 & 0.27 & 0.55 & 0.78 \\
POZ & 0.00 & 0.00 & 0.36 & 0.64 & 0.00 & 0.65 \\
LCJ & 0.00 & 1.00 & 0.00 & 0.00 & 0.00 & 0.26 \\
GDN & 0.00 & 0.00 & 0.00 & 0.00 & 1.00 & 1.00 \\
SZZ & 0.00 & 1.00 & 0.00 & 0.00 & 0.00 & 0.24 \\
BZG & 0.00 & 0.00 & 0.18 & 0.09 & 0.73 & 0.83 \\
RZE & 0.00 & 0.73 & 0.27 & 0.00 & 0.00 & 0.33 \\
IEG & 1.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.03 \\
\bottomrule
\end{tabular}



In [13]:
rank_superef = sorted(super_efficiency_scores.items(), key=lambda x: x[1], reverse=True)
rank_cross = cross_eff_matrix['CRavg'].sort_values(ascending=False)
rank_ee = df_efficiency['Efektywność'].sort_values(ascending=False)

print("Ranking superefektywności:")
print(" > ".join([dmu for dmu, score in rank_superef]))

print("\nRanking średniej efektywności krzyżowej:")
print(" > ".join(rank_cross.index))

print("\nRanking oczekiwanej wartości efektywności:")
print(" > ".join(rank_ee.index))


Ranking superefektywności:
WAW > GDN > BZG > KRK > WRO > POZ > KAT > RZE > LCJ > SZZ > IEG

Ranking średniej efektywności krzyżowej:
GDN > BZG > WAW > WRO > KRK > POZ > KAT > RZE > LCJ > SZZ > IEG

Ranking oczekiwanej wartości efektywności:
WAW > KRK > WRO > BZG > GDN > POZ > KAT > RZE > LCJ > SZZ > IEG


In [15]:
print("Ranking superefektywności:")
print(" \\succ ".join([dmu for dmu, score in rank_superef]))

print("\nRanking średniej efektywności krzyżowej:")
print(" \\succ ".join(rank_cross.index))

print("\nRanking oczekiwanej wartości efektywności:")
print(" \\succ ".join(rank_ee.index))

Ranking superefektywności:
WAW \succ GDN \succ BZG \succ KRK \succ WRO \succ POZ \succ KAT \succ RZE \succ LCJ \succ SZZ \succ IEG

Ranking średniej efektywności krzyżowej:
GDN \succ BZG \succ WAW \succ WRO \succ KRK \succ POZ \succ KAT \succ RZE \succ LCJ \succ SZZ \succ IEG

Ranking oczekiwanej wartości efektywności:
WAW \succ KRK \succ WRO \succ BZG \succ GDN \succ POZ \succ KAT \succ RZE \succ LCJ \succ SZZ \succ IEG
